In [1]:
import sys
sys.path.append("..")

import process_db
import h5py
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import re
import scanpy as sc
random.seed(1)

import pandas as pd

In [2]:
label_to_ids = pd.read_csv("/home/xcx/MYBenchmark-codes/4-scJoint/label_to_idx-uterus.txt", sep = ":", header = None)

index = {}
for i in range(label_to_ids.shape[0]):
    index[label_to_ids[0][i]] = label_to_ids[1][i]

In [4]:
lists_rna = []
lists_atac = []

label_rna = sc.read("/data1/xcx/unpaired-data/uterus/RNA/Uterus_Wang_2020_selected.h5ad", cache = True).obs["cell_type"].values
label_atac = sc.read("/data1/xcx/unpaired-data/uterus/ATAC/uterus_Zhang2021/adata_anno_selected.h5ad", cache = True).obs["celltype"].values

for line in label_rna:
    # print(line)
    if line == '0':
        continue
    lists_rna.append(index[line])

with open("/home/xcx/MYBenchmark-codes/4-scJoint/anno-num-uterus-rna.txt",'a') as f:
    for i in lists_rna:
        # print(i)
        f.write(str(i)+'\n')
        
for line in label_atac:
    # print(line)
    if line == '0':
        continue
    lists_atac.append(index[line])

with open("/home/xcx/MYBenchmark-codes/4-scJoint/anno-num-uterus-atac.txt",'a') as f:
    for i in lists_atac:
        # print(i)
        f.write(str(i)+'\n')

In [5]:
import pandas as pd
import scanpy as sc

df = sc.read("/data1/xcx/unpaired-data/uterus/highvar_region2gene.mtx", cache = True)
df = df.X
df = df.todense()
print(df.shape)

atac = sc.read("/data1/xcx/unpaired-data/uterus/ATAC/uterus_Zhang2021/highvar_counts_atac.mtx", cache = True)
atac = atac.X
atac = atac.todense()
print(atac.shape)

new_atac=np.matmul(atac,df)
print(new_atac.shape)

(32524, 1000)
(8314, 32524)
(8314, 1000)


In [6]:
# binary
atac_raw = new_atac
rna_raw = sc.read("/data1/xcx/unpaired-data/uterus/RNA/raw_highvar_counts_rna.mtx", cache = True)
rna_raw = rna_raw.X
rna_raw = rna_raw.todense()

counts_atac = np.where(atac_raw > 0, 1, 0)
counts_rna = np.where(rna_raw > 0, 1, 0)

import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import numpy as np
import scipy

mtx_rna = sparse.csr_matrix(counts_rna)
mtx_rna = sparse.coo_matrix(mtx_rna)
sio.mmwrite("/home/xcx/MYBenchmark-codes/4-scJoint/data/uterus/binary_counts_rna.mtx", mtx_rna)

mtx_atac = sparse.csr_matrix(counts_atac)
mtx_atac = sparse.coo_matrix(mtx_atac)
sio.mmwrite("/home/xcx/MYBenchmark-codes/4-scJoint/data/uterus/binary_counts_atac.mtx", mtx_atac)

In [2]:
run /home/xcx/MYBenchmark-codes/4-scJoint/main_uterus.py

Start time:  11:39:12
Training start [Stage1]
num_workers: 0
load mtx matrix: /home/xcx/MYBenchmark-codes/4-scJoint/data/uterus/binary_counts_rna.mtx
RNA
(8237, 1000)
(8237,)
load mtx matrix: /home/xcx/MYBenchmark-codes/4-scJoint/data/uterus/binary_counts_atac.mtx
ATAC
(8314, 1000)


/data1/xcx/anaconda3/envs/scDART/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


Epoch: 0
LR is set to 0.01
LR is set to 0.01
 [=============================>]    Step: 70ms | Tot: 3s306ms | encoding_loss: 8.593, rna_loss: 1.130 32/32   
Epoch: 1
 [=============================>]    Step: 165ms | Tot: 3s212ms | encoding_loss: 5.420, rna_loss: 0.750 32/32 
Epoch: 2
 [=============================>]    Step: 72ms | Tot: 2s998ms | encoding_loss: 5.032, rna_loss: 0.572 32/32  
Epoch: 3
 [=============================>]    Step: 97ms | Tot: 3s301ms | encoding_loss: 4.735, rna_loss: 0.475 32/32  
Epoch: 4
 [=============================>]    Step: 120ms | Tot: 3s230ms | encoding_loss: 4.570, rna_loss: 0.420 32/32 
Epoch: 5
 [=============================>]    Step: 67ms | Tot: 3s265ms | encoding_loss: 4.435, rna_loss: 0.380 32/32  
Epoch: 6
 [=============================>]    Step: 68ms | Tot: 3s294ms | encoding_loss: 4.377, rna_loss: 0.375 32/32  
Epoch: 7
 [=============================>]    Step: 91ms | Tot: 3s4ms | encoding_loss: 4.338, rna_loss: 0.350 32/32 32 
Epo

/data1/xcx/anaconda3/envs/scDART/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 2 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/xcx/MYBenchmark-codes/4-scJoint/util/closs.py:119: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1420.)
  distmat.addmm_(1, -2, x, self.centers.t())


 [=============================>]    Step: 94ms | Tot: 2s813ms | encoding_loss: 8.691, rna_loss: 2.132, center_loss: 5.037 32/32   
Epoch: 1
 [=============================>]    Step: 100ms | Tot: 3s176ms | encoding_loss: 5.195, rna_loss: 1.400, center_loss: 4.346 32/32 
Epoch: 2
 [=============================>]    Step: 88ms | Tot: 3s103ms | encoding_loss: 5.033, rna_loss: 1.032, center_loss: 3.819 32/32  
Epoch: 3
 [=============================>]    Step: 92ms | Tot: 3s154ms | encoding_loss: 4.827, rna_loss: 0.841, center_loss: 3.765 32/32  
Epoch: 4
 [=============================>]    Step: 88ms | Tot: 3s52ms | encoding_loss: 4.750, rna_loss: 0.747, center_loss: 3.663 32/32   
Epoch: 5
 [=============================>]    Step: 70ms | Tot: 3s23ms | encoding_loss: 4.675, rna_loss: 0.659, center_loss: 3.558 32/32   
Epoch: 6
 [=============================>]    Step: 93ms | Tot: 2s981ms | encoding_loss: 4.613, rna_loss: 0.626, center_loss: 3.521 32/32  
Epoch: 7
 [=================